# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
from pathlib import Path
from ScriptClasses import QualisysParser

## Load the xml file of the gait data 

In [2]:
database_path = Path(r'D:\Dokumente\UKSH_Programme\BIDS_Database')
uuid_path = Path(r'D:\Dokumente\UKSH_Programme\BIDS_Database\uuid.tsv')
#data_path = Path(r'/Users/marco/Documents/UKSH/Data')

#database_path = Path(r'/Users/marco/Documents/UKSH/BIDS_Database')
#uuid_path = Path(r'/Users/marco/Documents/UKSH/BIDS_Database/uuid.tsv')

qp = QualisysParser()

def select_file(filename):
    """_summary_

    Args:
        filename (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    file_path_list = []
    for root, dirs, files in os.walk(database_path):
        for file in files:
            if file.endswith(filename):
                file_path = os.path.join(root, file)
                file_path_list.append(file_path)
    return file_path_list




# Loading the timeseries Data 

In [3]:
file_path_list = select_file('session.tsv')
df_session = pd.read_csv(file_path_list[0], sep='\t')
df_session.set_index('index', inplace=True)
df_session.head(10)

,data
index,
Patient_ID,5cb8ce33-4b44-4840-9c33-05e5769db10f
First_name,NaN
Last_name,NaN
Date_of_birth,2000-01-01
Sex,unknown
Social_security_number,NaN
Reference,unspecified
Directory_pattern,$!Last name$_$!First name$_$Date of birth$_$!P...
Measurement_pattern,$TypeName$ $InstanceNumber$


In [4]:
# get Patient ID
id = df_session.at['Patient_ID', 'data']
sex = df_session.at['Sex', 'data']
dob = df_session.at['Date_of_birth', 'data']

In [5]:
file_path_list = select_file('timeseries.tsv')
for file_path in file_path_list:
    print(file_path.split('\\'))
    #print(file_path.split('/'))
    print('\n')
    

['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-002', 'gait', 'ses-001', 'sub-002_ses-001_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-003', 'gait', 'ses-002', 'sub-003_ses-002_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-004', 'gait', 'ses-001', 'sub-004_ses-001_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-004', 'gait', 'ses-002', 'sub-004_ses-002_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-005', 'gait', 'ses-001', 'sub-005_ses-001_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-007', 'gait', 'ses-001', 'sub-007_ses-001_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 'sub-008', 'gait', 'ses-001', 'sub-008_ses-001_timeseries.tsv']


['D:', 'Dokumente', 'UKSH_Programme', 'BIDS_Database', 'database', 's

In [6]:
# display all columns
pd.set_option('display.max_columns', None)
data = pd.DataFrame()
for file in file_path_list:
    data_raw = pd.read_csv(file, sep='\t')
    session_path = '\\'.join(file.split('\\')[:-1])
    #session_path = '/'.join(file.split('/')[:-1])
    for root, dirs, files2 in os.walk(session_path):
        for file2 in files2:
            if file2.endswith('session.tsv'):
                session_path = os.path.join(root, file2)
                df_session = pd.read_csv(session_path, sep='\t')
                df_session.set_index('index', inplace=True)
                id = df_session.at['Patient_ID', 'data']
                sex = df_session.at['Sex', 'data']
                dob = df_session.at['Date_of_birth', 'data']
                created = df_session.at['Creation_date', 'data'] + ' ' + df_session.at['Creation_time', 'data']

                subject = file.split('\\')[-4]
                #subject = file.split('/')[-4]

                """
                Just select data from Gait Markerless 1.c3d because this is the data from the actual gait analysis.
                Select the data where frames = 101
                Select just the data from type = DERIVED  Because the data got computed with additional calculations
                Select just the data from folder = PROCESSED because this ist the data which got additionaly processed
                """
                
                # print(f'Before filter: {data_raw.shape}')
                data_raw = data_raw.loc[(data_raw['frames'] == 101) & (data_raw['type'].str.contains('DERIVED')) & (data_raw['owner'].str.contains('Gait Markerless 1'))]
                # print(f'After filter: {data_raw.shape}')
                #display(data_raw)
                data_raw.drop(labels=['owner', 'type', 'frames', 'folder', 'Frame_Start', 'Frame_End', 'Time_End', 'Event_Sequence'], axis=1, inplace=True)
                
                data_raw['name'] = data_raw['name'] + '_' + data_raw['component_value']
                data_raw.drop(labels=['component_value'], axis=1, inplace=True)

               
                # Spalten 'name' und 'data' extrahieren
                data_raw = data_raw[['name', 'data']].copy()

                # 'data' Spalte in Listen umwandeln
                data_raw['data'] = data_raw['data'].apply(lambda data: [float(x) for x in data.split(',') if x != 'nodata'])
                # Temporäres DataFrame erstellen
                df_temp = pd.DataFrame()
                for name in data_raw['name'].unique():
                    temp = data_raw.loc[data_raw['name'] == name].copy()
                    temp['cycle'] = [str(i) for i in range(1, len(temp)+1)]
                    df_temp = pd.concat([df_temp, temp], axis=0)
                #display(df_temp)
                # Pivot-Tabelle erstellen
                try:
                    data_raw = df_temp.pivot(index='cycle', columns='name', values='data').sort_index().reset_index(drop=False)
                    data_raw['cycle'] = data_raw['cycle'].apply(lambda x: f'cycle_{x}')


                    # data_raw = data_raw.set_index('name').T
                    
                    data_raw['subject'] = subject
                    data_raw['Patient_ID'] = id
                    data_raw['sex'] = sex
                    data_raw['dob'] = dob
                    data_raw['created'] = created
                    

                    
                    
                    data = pd.concat([data, data_raw], axis=0, ignore_index=True)
                except:
                    continue
                #display(data)
               
        
                


In [7]:
# bring the last columns, subject	Patient_ID	sex	dob	created to the front
cols = data.columns.tolist()
cols = cols[-5:] + cols[:-5]
data = data[cols]

In [8]:
data.head(3)

name,subject,Patient_ID,sex,dob,created,cycle,Left Ankle Angles_X,Left Ankle Angles_Y,Left Ankle Angles_Z,Left CenterOfMass_corr_X,Left CenterOfMass_corr_Y,Left CenterOfMass_corr_Z,Left Elbow Angles_X,Left Elbow Angles_Y,Left Elbow Angles_Z,Left Foot Pitch Angles_X,Left Foot Pitch Angles_Y,Left Foot Pitch Angles_Z,Left Foot Progression_X,Left Foot Progression_Y,Left Foot Progression_Z,Left Hip Angles_X,Left Hip Angles_Y,Left Hip Angles_Z,Left Knee Angles_X,Left Knee Angles_Y,Left Knee Angles_Z,Left Pelvic Angles_X,Left Pelvic Angles_Y,Left Pelvic Angles_Z,Left Shoulder Angles_X,Left Shoulder Angles_Y,Left Shoulder Angles_Z,Left Thorax Angles_X,Left Thorax Angles_Y,Left Thorax Angles_Z,Left Thorax_Lab Angles_X,Left Thorax_Lab Angles_Y,Left Thorax_Lab Angles_Z,Right Ankle Angles_X,Right Ankle Angles_Y,Right Ankle Angles_Z,Right CenterOfMass_corr_X,Right CenterOfMass_corr_Y,Right CenterOfMass_corr_Z,Right Elbow Angles_X,Right Elbow Angles_Y,Right Elbow Angles_Z,Right Foot Pitch Angles_X,Right Foot Pitch Angles_Y,Right Foot Pitch Angles_Z,Right Foot Progression_X,Right Foot Progression_Y,Right Foot Progression_Z,Right Hip Angles_X,Right Hip Angles_Y,Right Hip Angles_Z,Right Knee Angles_X,Right Knee Angles_Y,Right Knee Angles_Z,Right Pelvic Angles_X,Right Pelvic Angles_Y,Right Pelvic Angles_Z,Right Shoulder Angles_X,Right Shoulder Angles_Y,Right Shoulder Angles_Z,Right Thorax Angles_X,Right Thorax Angles_Y,Right Thorax Angles_Z,Right Thorax_Lab Angles_X,Right Thorax_Lab Angles_Y,Right Thorax_Lab Angles_Z
0,sub-002,3b7b78ed-042a-4316-b4ae-9fc8cbb9359e,male,2005-02-10,2024-05-29 12:27:14,cycle_1,"[-2.768442, -3.496685, -4.069201, -4.462231, -...","[7.935368, 8.292075, 8.685886, 9.081171, 9.438...","[0.602269, 0.629934, 0.660829, 0.667538, 0.672...","[0.0, -0.002226, -0.00464, -0.007243, -0.01002...","[0.0, -0.000268, -0.000407, -0.00043, -0.00035...","[0.0, -0.001321, -0.002628, -0.003877, -0.0050...","[22.490803, 22.188433, 21.868492, 21.554268, 2...","[-0.010252, -0.01488, -0.017207, -0.016572, -0...","[104.20636, 103.466881, 102.595413, 101.67038,...","[14.214731, 13.85244, 13.169363, 12.26093, 11....","[-3.179904, -3.617692, -4.20662, -4.875635, -5...","[19.032904, 18.2679, 17.341208, 16.317291, 15....","[14.214735, 13.852443, 13.169367, 12.260934, 1...","[-3.179903, -3.61769, -4.206618, -4.875634, -5...","[-19.032904, -18.2679, -17.341208, -16.317291,...","[20.6766, 20.222738, 19.856031, 19.556093, 19....","[-4.393382, -4.227548, -4.138162, -4.11265, -4...","[-0.280891, -1.543496, -2.484963, -3.042148, -...","[2.135388, 1.461672, 1.318257, 1.61889, 2.2538...","[-0.194929, -0.818533, -1.206393, -1.397462, -...","[-14.350356, -12.79824, -11.413175, -10.301519...","[3.991429, 3.891927, 3.795211, 3.698814, 3.599...","[-0.522435, -0.717429, -0.903394, -1.072962, -...","[-4.647504, -4.119165, -3.555717, -2.977561, -...","[2.708186, 2.722901, 2.779783, 2.872899, 2.991...","[-8.669715, -8.55989, -8.467485, -8.390489, -8...","[16.503153, 16.36441, 16.326563, 16.373936, 16...","[3.141546, 3.062634, 2.954683, 2.827985, 2.697...","[-1.945191, -1.821488, -1.716818, -1.627697, -...","[-2.919237, -3.102238, -3.302948, -3.50874, -3...","[7.254066, 7.08643, 6.894266, 6.684382, 6.4677...","[-2.086775, -2.146077, -2.215677, -2.286167, -...","[-7.787728, -7.421407, -7.040127, -6.651467, -...","[-5.482529, -5.833911, -6.063342, -6.161349, -...","[4.380441, 4.43795, 4.501745, 4.57004, 4.64473...","[2.074525, 3.183789, 4.164791, 4.954755, 5.505...","[0.0, 0.00154, 0.003264, 0.00518, 0.007285, 0....","[0.0, 0.000522, 0.001184, 0.001996, 0.002956, ...","[0.0, -0.000998, -0.002004, -0.002973, -0.0038...","[28.822729, 28.978001, 29.148277, 29.292614, 2...","[0.003681, 0.007298, 0.009037, 0.008681, 0.007...","[97.213318, 96.508972, 95.770767, 95.081993, 9...","[10.846501, 9.729046, 8.539912, 7.347647, 6.20...","[4.045348, 4.578834, 5.088349, 5.536511, 5.904...","[-13.617139, -12.353103, -10.997869, -9.585029...","[10.846505, 9.72905, 8.539917, 7.347651, 6.

In [9]:
uuid = pd.read_csv(uuid_path, sep='\t')

data = data.merge(uuid, how='left', left_on='subject', right_on='subject').drop(labels=['patient', 'Patient_ID', 'id'], axis=1)

In [10]:
# just select data of men and female 

data = data.loc[(data.sex == 'male') | (data.sex == 'female')]

# reorder columns

data.head()

,subject,sex,dob,created,cycle,Left Ankle Angles_X,Left Ankle Angles_Y,Left Ankle Angles_Z,Left CenterOfMass_corr_X,Left CenterOfMass_corr_Y,Left CenterOfMass_corr_Z,Left Elbow Angles_X,Left Elbow Angles_Y,Left Elbow Angles_Z,Left Foot Pitch Angles_X,Left Foot Pitch Angles_Y,Left Foot Pitch Angles_Z,Left Foot Progression_X,Left Foot Progression_Y,Left Foot Progression_Z,Left Hip Angles_X,Left Hip Angles_Y,Left Hip Angles_Z,Left Knee Angles_X,Left Knee Angles_Y,Left Knee Angles_Z,Left Pelvic Angles_X,Left Pelvic Angles_Y,Left Pelvic Angles_Z,Left Shoulder Angles_X,Left Shoulder Angles_Y,Left Shoulder Angles_Z,Left Thorax Angles_X,Left Thorax Angles_Y,Left Thorax Angles_Z,Left Thorax_Lab Angles_X,Left Thorax_Lab Angles_Y,Left Thorax_Lab Angles_Z,Right Ankle Angles_X,Right Ankle Angles_Y,Right Ankle Angles_Z,Right CenterOfMass_corr_X,Right CenterOfMass_corr_Y,Right CenterOfMass_corr_Z,Right Elbow Angles_X,Right Elbow Angles_Y,Right Elbow Angles_Z,Right Foot Pitch Angles_X,Right Foot Pitch Angles_Y,Right Foot Pitch Angles_Z,Right Foot Progression_X,Right Foot Progression_Y,Right Foot Progression_Z,Right Hip Angles_X,Right Hip Angles_Y,Right Hip Angles_Z,Right Knee Angles_X,Right Knee Angles_Y,Right Knee Angles_Z,Right Pelvic Angles_X,Right Pelvic Angles_Y,Right Pelvic Angles_Z,Right Shoulder Angles_X,Right Shoulder Angles_Y,Right Shoulder Angles_Z,Right Thorax Angles_X,Right Thorax Angles_Y,Right Thorax Angles_Z,Right Thorax_Lab Angles_X,Right Thorax_Lab Angles_Y,Right Thorax_Lab Angles_Z
0,sub-002,male,2005-02-10,2024-05-29 12:27:14,cycle_1,"[-2.768442, -3.496685, -4.069201, -4.462231, -...","[7.935368, 8.292075, 8.685886, 9.081171, 9.438...","[0.602269, 0.629934, 0.660829, 0.667538, 0.672...","[0.0, -0.002226, -0.00464, -0.007243, -0.01002...","[0.0, -0.000268, -0.000407, -0.00043, -0.00035...","[0.0, -0.001321, -0.002628, -0.003877, -0.0050...","[22.490803, 22.188433, 21.868492, 21.554268, 2...","[-0.010252, -0.01488, -0.017207, -0.016572, -0...","[104.20636, 103.466881, 102.595413, 101.67038,...","[14.214731, 13.85244, 13.169363, 12.26093, 11....","[-3.179904, -3.617692, -4.20662, -4.875635, -5...","[19.032904, 18.2679, 17.341208, 16.317291, 15....","[14.214735, 13.852443, 13.169367, 12.260934, 1...","[-3.179903, -3.61769, -4.206618, -4.875634, -5...","[-19.032904, -18.2679, -17.341208, -16.317291,...","[20.6766, 20.222738, 19.856031, 19.556093, 19....","[-4.393382, -4.227548, -4.138162, -4.11265, -4...","[-0.280891, -1.543496, -2.484963, -3.042148, -...","[2.135388, 1.461672, 1.318257, 1.61889, 2.2538...","[-0.194929, -0.818533, -1.206393, -1.397462, -...","[-14.350356, -12.79824, -11.413175, -10.301519...","[3.991429, 3.891927, 3.795211, 3.698814, 3.599...","[-0.522435, -0.717429, -0.903394, -1.072962, -...","[-4.647504, -4.119165, -3.555717, -2.977561, -...","[2.708186, 2.722901, 2.779783, 2.872899, 2.991...","[-8.669715, -8.55989, -8.467485, -8.390489, -8...","[16.503153, 16.36441, 16.326563, 16.373936, 16...","[3.141546, 3.062634, 2.954683, 2.827985, 2.697...","[-1.945191, -1.821488, -1.716818, -1.627697, -...","[-2.919237, -3.102238, -3.302948, -3.50874, -3...","[7.254066, 7.08643, 6.894266, 6.684382, 6.4677...","[-2.086775, -2.146077, -2.215677, -2.286167, -...","[-7.787728, -7.421407, -7.040127, -6.651467, -...","[-5.482529, -5.833911, -6.063342, -6.161349, -...","[4.380441, 4.43795, 4.501745, 4.57004, 4.64473...","[2.074525, 3.183789, 4.164791, 4.954755, 5.505...","[0.0, 0.00154, 0.003264, 0.00518, 0.007285, 0....","[0.0, 0.000522, 0.001184, 0.001996, 0.002956, ...","[0.0, -0.000998, -0.002004, -0.002973, -0.0038...","[28.822729, 28.978001, 29.148277, 29.292614, 2...","[0.003681, 0.007298, 0.009037, 0.008681, 0.007...","[97.213318, 96.508972, 95.770767, 95.081993, 9...","[10.846501, 9.729046, 8.539912, 7.347647, 6.20...","[4.045348, 4.578834, 5.088349, 5.536511, 5.904...","[-13.617139, -12.353103, -10.997869, -9.585029...","[10.846505, 9.72905, 8.539917, 7.347651, 6.206...","[4.045347, 4.578834, 5.088348, 5.53651, 5.9

In [11]:
data_clean = data.dropna(axis=0, how='any')
data_clean['created_new'] = pd.to_datetime(data_clean['created'], format='%Y-%m-%d %H:%M:%S')
data_clean['created_new'] = data_clean['created_new'].dt.date
data_clean['dob'] = pd.to_datetime(data_clean['dob'], format='%Y-%m-%d').dt.date

data_clean['age'] = data_clean['created_new'] - data_clean['dob']

# format age to years
data_clean['age'] = data_clean['age'].apply(lambda x: int(x.days/365))
data_clean.drop(labels=['created_new', 'dob'], axis=1, inplace=True)
data_clean.head()

C:\Users\Gener\AppData\Local\Temp\ipykernel_11752\199344160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['created_new'] = pd.to_datetime(data_clean['created'], format='%Y-%m-%d %H:%M:%S')
C:\Users\Gener\AppData\Local\Temp\ipykernel_11752\199344160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['created_new'] = data_clean['created_new'].dt.date
C:\Users\Gener\AppData\Local\Temp\ipykernel_11752\199344160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,subject,sex,created,cycle,Left Ankle Angles_X,Left Ankle Angles_Y,Left Ankle Angles_Z,Left CenterOfMass_corr_X,Left CenterOfMass_corr_Y,Left CenterOfMass_corr_Z,Left Elbow Angles_X,Left Elbow Angles_Y,Left Elbow Angles_Z,Left Foot Pitch Angles_X,Left Foot Pitch Angles_Y,Left Foot Pitch Angles_Z,Left Foot Progression_X,Left Foot Progression_Y,Left Foot Progression_Z,Left Hip Angles_X,Left Hip Angles_Y,Left Hip Angles_Z,Left Knee Angles_X,Left Knee Angles_Y,Left Knee Angles_Z,Left Pelvic Angles_X,Left Pelvic Angles_Y,Left Pelvic Angles_Z,Left Shoulder Angles_X,Left Shoulder Angles_Y,Left Shoulder Angles_Z,Left Thorax Angles_X,Left Thorax Angles_Y,Left Thorax Angles_Z,Left Thorax_Lab Angles_X,Left Thorax_Lab Angles_Y,Left Thorax_Lab Angles_Z,Right Ankle Angles_X,Right Ankle Angles_Y,Right Ankle Angles_Z,Right CenterOfMass_corr_X,Right CenterOfMass_corr_Y,Right CenterOfMass_corr_Z,Right Elbow Angles_X,Right Elbow Angles_Y,Right Elbow Angles_Z,Right Foot Pitch Angles_X,Right Foot Pitch Angles_Y,Right Foot Pitch Angles_Z,Right Foot Progression_X,Right Foot Progression_Y,Right Foot Progression_Z,Right Hip Angles_X,Right Hip Angles_Y,Right Hip Angles_Z,Right Knee Angles_X,Right Knee Angles_Y,Right Knee Angles_Z,Right Pelvic Angles_X,Right Pelvic Angles_Y,Right Pelvic Angles_Z,Right Shoulder Angles_X,Right Shoulder Angles_Y,Right Shoulder Angles_Z,Right Thorax Angles_X,Right Thorax Angles_Y,Right Thorax Angles_Z,Right Thorax_Lab Angles_X,Right Thorax_Lab Angles_Y,Right Thorax_Lab Angles_Z,age
0,sub-002,male,2024-05-29 12:27:14,cycle_1,"[-2.768442, -3.496685, -4.069201, -4.462231, -...","[7.935368, 8.292075, 8.685886, 9.081171, 9.438...","[0.602269, 0.629934, 0.660829, 0.667538, 0.672...","[0.0, -0.002226, -0.00464, -0.007243, -0.01002...","[0.0, -0.000268, -0.000407, -0.00043, -0.00035...","[0.0, -0.001321, -0.002628, -0.003877, -0.0050...","[22.490803, 22.188433, 21.868492, 21.554268, 2...","[-0.010252, -0.01488, -0.017207, -0.016572, -0...","[104.20636, 103.466881, 102.595413, 101.67038,...","[14.214731, 13.85244, 13.169363, 12.26093, 11....","[-3.179904, -3.617692, -4.20662, -4.875635, -5...","[19.032904, 18.2679, 17.341208, 16.317291, 15....","[14.214735, 13.852443, 13.169367, 12.260934, 1...","[-3.179903, -3.61769, -4.206618, -4.875634, -5...","[-19.032904, -18.2679, -17.341208, -16.317291,...","[20.6766, 20.222738, 19.856031, 19.556093, 19....","[-4.393382, -4.227548, -4.138162, -4.11265, -4...","[-0.280891, -1.543496, -2.484963, -3.042148, -...","[2.135388, 1.461672, 1.318257, 1.61889, 2.2538...","[-0.194929, -0.818533, -1.206393, -1.397462, -...","[-14.350356, -12.79824, -11.413175, -10.301519...","[3.991429, 3.891927, 3.795211, 3.698814, 3.599...","[-0.522435, -0.717429, -0.903394, -1.072962, -...","[-4.647504, -4.119165, -3.555717, -2.977561, -...","[2.708186, 2.722901, 2.779783, 2.872899, 2.991...","[-8.669715, -8.55989, -8.467485, -8.390489, -8...","[16.503153, 16.36441, 16.326563, 16.373936, 16...","[3.141546, 3.062634, 2.954683, 2.827985, 2.697...","[-1.945191, -1.821488, -1.716818, -1.627697, -...","[-2.919237, -3.102238, -3.302948, -3.50874, -3...","[7.254066, 7.08643, 6.894266, 6.684382, 6.4677...","[-2.086775, -2.146077, -2.215677, -2.286167, -...","[-7.787728, -7.421407, -7.040127, -6.651467, -...","[-5.482529, -5.833911, -6.063342, -6.161349, -...","[4.380441, 4.43795, 4.501745, 4.57004, 4.64473...","[2.074525, 3.183789, 4.164791, 4.954755, 5.505...","[0.0, 0.00154, 0.003264, 0.00518, 0.007285, 0....","[0.0, 0.000522, 0.001184, 0.001996, 0.002956, ...","[0.0, -0.000998, -0.002004, -0.002973, -0.0038...","[28.822729, 28.978001, 29.148277, 29.292614, 2...","[0.003681, 0.007298, 0.009037, 0.008681, 0.007...","[97.213318, 96.508972, 95.770767, 95.081993, 9...","[10.846501, 9.729046, 8.539912, 7.347647, 6.20...","[4.045348, 4.578834, 5.088349, 5.536511, 5.904...","[-13.617139, -12.353103, -10.997869, -9.585029...","[10.846505, 9.72905, 8.539917, 7.347651, 6.206...","[4.045347, 4.578834, 5.088348, 5.53651, 5.9045...","[-

In [12]:
# bring age column to the front
cols = data_clean.columns.tolist()
cols = cols[-1:] + cols[:-1]
data_clean = data_clean[cols]
data_clean.head()

,age,subject,sex,created,cycle,Left Ankle Angles_X,Left Ankle Angles_Y,Left Ankle Angles_Z,Left CenterOfMass_corr_X,Left CenterOfMass_corr_Y,Left CenterOfMass_corr_Z,Left Elbow Angles_X,Left Elbow Angles_Y,Left Elbow Angles_Z,Left Foot Pitch Angles_X,Left Foot Pitch Angles_Y,Left Foot Pitch Angles_Z,Left Foot Progression_X,Left Foot Progression_Y,Left Foot Progression_Z,Left Hip Angles_X,Left Hip Angles_Y,Left Hip Angles_Z,Left Knee Angles_X,Left Knee Angles_Y,Left Knee Angles_Z,Left Pelvic Angles_X,Left Pelvic Angles_Y,Left Pelvic Angles_Z,Left Shoulder Angles_X,Left Shoulder Angles_Y,Left Shoulder Angles_Z,Left Thorax Angles_X,Left Thorax Angles_Y,Left Thorax Angles_Z,Left Thorax_Lab Angles_X,Left Thorax_Lab Angles_Y,Left Thorax_Lab Angles_Z,Right Ankle Angles_X,Right Ankle Angles_Y,Right Ankle Angles_Z,Right CenterOfMass_corr_X,Right CenterOfMass_corr_Y,Right CenterOfMass_corr_Z,Right Elbow Angles_X,Right Elbow Angles_Y,Right Elbow Angles_Z,Right Foot Pitch Angles_X,Right Foot Pitch Angles_Y,Right Foot Pitch Angles_Z,Right Foot Progression_X,Right Foot Progression_Y,Right Foot Progression_Z,Right Hip Angles_X,Right Hip Angles_Y,Right Hip Angles_Z,Right Knee Angles_X,Right Knee Angles_Y,Right Knee Angles_Z,Right Pelvic Angles_X,Right Pelvic Angles_Y,Right Pelvic Angles_Z,Right Shoulder Angles_X,Right Shoulder Angles_Y,Right Shoulder Angles_Z,Right Thorax Angles_X,Right Thorax Angles_Y,Right Thorax Angles_Z,Right Thorax_Lab Angles_X,Right Thorax_Lab Angles_Y,Right Thorax_Lab Angles_Z
0,19,sub-002,male,2024-05-29 12:27:14,cycle_1,"[-2.768442, -3.496685, -4.069201, -4.462231, -...","[7.935368, 8.292075, 8.685886, 9.081171, 9.438...","[0.602269, 0.629934, 0.660829, 0.667538, 0.672...","[0.0, -0.002226, -0.00464, -0.007243, -0.01002...","[0.0, -0.000268, -0.000407, -0.00043, -0.00035...","[0.0, -0.001321, -0.002628, -0.003877, -0.0050...","[22.490803, 22.188433, 21.868492, 21.554268, 2...","[-0.010252, -0.01488, -0.017207, -0.016572, -0...","[104.20636, 103.466881, 102.595413, 101.67038,...","[14.214731, 13.85244, 13.169363, 12.26093, 11....","[-3.179904, -3.617692, -4.20662, -4.875635, -5...","[19.032904, 18.2679, 17.341208, 16.317291, 15....","[14.214735, 13.852443, 13.169367, 12.260934, 1...","[-3.179903, -3.61769, -4.206618, -4.875634, -5...","[-19.032904, -18.2679, -17.341208, -16.317291,...","[20.6766, 20.222738, 19.856031, 19.556093, 19....","[-4.393382, -4.227548, -4.138162, -4.11265, -4...","[-0.280891, -1.543496, -2.484963, -3.042148, -...","[2.135388, 1.461672, 1.318257, 1.61889, 2.2538...","[-0.194929, -0.818533, -1.206393, -1.397462, -...","[-14.350356, -12.79824, -11.413175, -10.301519...","[3.991429, 3.891927, 3.795211, 3.698814, 3.599...","[-0.522435, -0.717429, -0.903394, -1.072962, -...","[-4.647504, -4.119165, -3.555717, -2.977561, -...","[2.708186, 2.722901, 2.779783, 2.872899, 2.991...","[-8.669715, -8.55989, -8.467485, -8.390489, -8...","[16.503153, 16.36441, 16.326563, 16.373936, 16...","[3.141546, 3.062634, 2.954683, 2.827985, 2.697...","[-1.945191, -1.821488, -1.716818, -1.627697, -...","[-2.919237, -3.102238, -3.302948, -3.50874, -3...","[7.254066, 7.08643, 6.894266, 6.684382, 6.4677...","[-2.086775, -2.146077, -2.215677, -2.286167, -...","[-7.787728, -7.421407, -7.040127, -6.651467, -...","[-5.482529, -5.833911, -6.063342, -6.161349, -...","[4.380441, 4.43795, 4.501745, 4.57004, 4.64473...","[2.074525, 3.183789, 4.164791, 4.954755, 5.505...","[0.0, 0.00154, 0.003264, 0.00518, 0.007285, 0....","[0.0, 0.000522, 0.001184, 0.001996, 0.002956, ...","[0.0, -0.000998, -0.002004, -0.002973, -0.0038...","[28.822729, 28.978001, 29.148277, 29.292614, 2...","[0.003681, 0.007298, 0.009037, 0.008681, 0.007...","[97.213318, 96.508972, 95.770767, 95.081993, 9...","[10.846501, 9.729046, 8.539912, 7.347647, 6.20...","[4.045348, 4.578834, 5.088349, 5.536511, 5.904...","[-13.617139, -12.353103, -10.997869, -9.585029...","[10.846505, 9.72905, 8.539917, 7.347651, 6.206...","[4.045347, 4.578834, 5.088348, 5.53651, 5.9045...",

In [13]:
# check len of each list in columns and drop rows where len is not 101
# Filter rows where all lists have length 101
print(f'shape before: {data_clean.shape}')
columns_to_check = data_clean.columns[5:-1]
df_filtered = data_clean[data_clean[columns_to_check].map(lambda x: len(x) == 101).all(axis=1)]
print(f'shape after: {df_filtered.shape}')

shape before: (3845, 71)
shape after: (3656, 71)


In [14]:
df_filtered['Left Ankle Angles_X'].map(len)

0       101
1       101
2       101
3       101
4       101
       ... 
7957    101
7968    101
7979    101
7985    101
7986    101
Name: Left Ankle Angles_X, Length: 3656, dtype: int64

In [15]:
df_filtered.head(2)

,age,subject,sex,created,cycle,Left Ankle Angles_X,Left Ankle Angles_Y,Left Ankle Angles_Z,Left CenterOfMass_corr_X,Left CenterOfMass_corr_Y,Left CenterOfMass_corr_Z,Left Elbow Angles_X,Left Elbow Angles_Y,Left Elbow Angles_Z,Left Foot Pitch Angles_X,Left Foot Pitch Angles_Y,Left Foot Pitch Angles_Z,Left Foot Progression_X,Left Foot Progression_Y,Left Foot Progression_Z,Left Hip Angles_X,Left Hip Angles_Y,Left Hip Angles_Z,Left Knee Angles_X,Left Knee Angles_Y,Left Knee Angles_Z,Left Pelvic Angles_X,Left Pelvic Angles_Y,Left Pelvic Angles_Z,Left Shoulder Angles_X,Left Shoulder Angles_Y,Left Shoulder Angles_Z,Left Thorax Angles_X,Left Thorax Angles_Y,Left Thorax Angles_Z,Left Thorax_Lab Angles_X,Left Thorax_Lab Angles_Y,Left Thorax_Lab Angles_Z,Right Ankle Angles_X,Right Ankle Angles_Y,Right Ankle Angles_Z,Right CenterOfMass_corr_X,Right CenterOfMass_corr_Y,Right CenterOfMass_corr_Z,Right Elbow Angles_X,Right Elbow Angles_Y,Right Elbow Angles_Z,Right Foot Pitch Angles_X,Right Foot Pitch Angles_Y,Right Foot Pitch Angles_Z,Right Foot Progression_X,Right Foot Progression_Y,Right Foot Progression_Z,Right Hip Angles_X,Right Hip Angles_Y,Right Hip Angles_Z,Right Knee Angles_X,Right Knee Angles_Y,Right Knee Angles_Z,Right Pelvic Angles_X,Right Pelvic Angles_Y,Right Pelvic Angles_Z,Right Shoulder Angles_X,Right Shoulder Angles_Y,Right Shoulder Angles_Z,Right Thorax Angles_X,Right Thorax Angles_Y,Right Thorax Angles_Z,Right Thorax_Lab Angles_X,Right Thorax_Lab Angles_Y,Right Thorax_Lab Angles_Z
0,19,sub-002,male,2024-05-29 12:27:14,cycle_1,"[-2.768442, -3.496685, -4.069201, -4.462231, -...","[7.935368, 8.292075, 8.685886, 9.081171, 9.438...","[0.602269, 0.629934, 0.660829, 0.667538, 0.672...","[0.0, -0.002226, -0.00464, -0.007243, -0.01002...","[0.0, -0.000268, -0.000407, -0.00043, -0.00035...","[0.0, -0.001321, -0.002628, -0.003877, -0.0050...","[22.490803, 22.188433, 21.868492, 21.554268, 2...","[-0.010252, -0.01488, -0.017207, -0.016572, -0...","[104.20636, 103.466881, 102.595413, 101.67038,...","[14.214731, 13.85244, 13.169363, 12.26093, 11....","[-3.179904, -3.617692, -4.20662, -4.875635, -5...","[19.032904, 18.2679, 17.341208, 16.317291, 15....","[14.214735, 13.852443, 13.169367, 12.260934, 1...","[-3.179903, -3.61769, -4.206618, -4.875634, -5...","[-19.032904, -18.2679, -17.341208, -16.317291,...","[20.6766, 20.222738, 19.856031, 19.556093, 19....","[-4.393382, -4.227548, -4.138162, -4.11265, -4...","[-0.280891, -1.543496, -2.484963, -3.042148, -...","[2.135388, 1.461672, 1.318257, 1.61889, 2.2538...","[-0.194929, -0.818533, -1.206393, -1.397462, -...","[-14.350356, -12.79824, -11.413175, -10.301519...","[3.991429, 3.891927, 3.795211, 3.698814, 3.599...","[-0.522435, -0.717429, -0.903394, -1.072962, -...","[-4.647504, -4.119165, -3.555717, -2.977561, -...","[2.708186, 2.722901, 2.779783, 2.872899, 2.991...","[-8.669715, -8.55989, -8.467485, -8.390489, -8...","[16.503153, 16.36441, 16.326563, 16.373936, 16...","[3.141546, 3.062634, 2.954683, 2.827985, 2.697...","[-1.945191, -1.821488, -1.716818, -1.627697, -...","[-2.919237, -3.102238, -3.302948, -3.50874, -3...","[7.254066, 7.08643, 6.894266, 6.684382, 6.4677...","[-2.086775, -2.146077, -2.215677, -2.286167, -...","[-7.787728, -7.421407, -7.040127, -6.651467, -...","[-5.482529, -5.833911, -6.063342, -6.161349, -...","[4.380441, 4.43795, 4.501745, 4.57004, 4.64473...","[2.074525, 3.183789, 4.164791, 4.954755, 5.505...","[0.0, 0.00154, 0.003264, 0.00518, 0.007285, 0....","[0.0, 0.000522, 0.001184, 0.001996, 0.002956, ...","[0.0, -0.000998, -0.002004, -0.002973, -0.0038...","[28.822729, 28.978001, 29.148277, 29.292614, 2...","[0.003681, 0.007298, 0.009037, 0.008681, 0.007...","[97.213318, 96.508972, 95.770767, 95.081993, 9...","[10.846501, 9.729046, 8.539912, 7.347647, 6.20...","[4.045348, 4.578834, 5.088349, 5.536511, 5.904...","[-13.617139, -12.353103, -10.997869, -9.585029...","[10.846505, 9.72905, 8.539917, 7.347651, 6.206...","[4.045347, 4.578834, 5.088348, 5.53651, 5.9045...",

In [16]:
df_filtered.reset_index(drop=True, inplace=True)

In [17]:
df_filtered.to_csv(r'../data/data_filtered.csv', sep=';', index=True)

In [18]:
df_filtered['Left Ankle Angles_X'].map(len)

0       101
1       101
2       101
3       101
4       101
       ... 
3651    101
3652    101
3653    101
3654    101
3655    101
Name: Left Ankle Angles_X, Length: 3656, dtype: int64

In [21]:
for col in df_filtered['Left Ankle Angles_X']:
    print(col)
    print(len(col))
    break

[-2.768442, -3.496685, -4.069201, -4.462231, -4.666633, -4.682414, -4.515592, -4.180984, -3.706594, -3.139118, -2.537701, -1.963964, -1.466303, -1.066267, -0.756787, -0.504265, -0.261652, 0.01344, 0.34754, 0.742804, 1.182916, 1.641068, 2.08797, 2.49984, 2.859748, 3.156221, 3.378663, 3.515935, 3.559311, 3.507754, 3.37387, 3.184321, 2.978447, 2.80306, 2.702815, 2.711922, 2.844483, 3.088517, 3.410834, 3.764388, 4.103315, 4.394764, 4.623485, 4.793108, 4.918473, 5.018241, 5.109294, 5.203904, 5.311924, 5.442635, 5.60623, 5.810938, 6.058756, 6.340917, 6.635428, 6.912546, 7.140619, 7.294051, 7.353762, 7.304969, 7.135235, 6.837193, 6.410499, 5.862523, 5.207317, 4.464396, 3.656571, 2.806496, 1.930532, 1.038359, 0.13659, -0.759186, -1.617855, -2.396062, -3.047052, -3.526333, -3.794313, -3.822126, -3.608169, -3.178462, -2.578577, -1.857819, -1.060251, -0.225511, 0.608542, 1.403128, 2.11709, 2.705072, 3.115066, 3.293783, 3.197195, 2.803841, 2.127432, 1.21593, 0.146981, -0.984128, -2.083462, -3.0710

In [22]:
df_filtered.to_json(r'../data/data_filtered.json', orient='records')